In [109]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [110]:
from machine import Pin, I2C

i2c = I2C(scl=Pin(15), sda=Pin(4), freq=450000)
print(list(map(hex, i2c.scan())))
i2c.writeto_mem(0x29, 0x80, b'\x03')
print("chipid", hex(i2c.readfrom_mem(0x29, 0x92, 1)[0]))

i2c.writeto_mem(0x29, 0x8F, b'\x00')  # gain of 1x
i2c.writeto_mem(0x29, 0x81, b'\x00')  # integration time 700ms

i2c.writeto_mem(0x29, 0x8F, b'\x01')  # gain of 2x
i2c.writeto_mem(0x29, 0x81, b'\xC0')  # integration time 154ms

i2c.writeto_mem(0x29, 0x8F, b'\x02')  # gain of 4x
i2c.writeto_mem(0x29, 0x81, b'\xEB')  # integration time 50ms


['0x29']
chipid 0x44


In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [3]:
x = i2c.readfrom_mem(0x29, 0x94, 8)
print(x)

b'\xc6\x07\xcb\x01\xda\x02\xb9\x02'


In [4]:
print(x[0] + x[1]*256)

1990


In [111]:
import struct, time
t0 = time.ticks_ms()
for i in range(20):
    print(time.ticks_ms() - t0, struct.unpack("<HHHH", i2c.readfrom_mem(0x29, 0x94, 8)))
    time.sleep(1)

0 (334, 98, 116, 102)
1002 (387, 110, 139, 129)
2002 (386, 109, 139, 129)
3003 (385, 109, 139, 129)
4004 (385, 109, 138, 129)
5005 (385, 109, 138, 129)
6006 (385, 109, 139, 129)
7006 (386, 109, 139, 129)
8007 (384, 109, 138, 129)
9008 (386, 109, 139, 129)
.10009 (386, 109, 139, 129)
11009 (385, 109, 138, 129)
12010 (384, 109, 138, 128)
13011 (326, 95, 114, 100)
14012 (331, 97, 115, 101)
15012 (331, 97, 115, 101)
16013 (331, 97, 115, 101)
17014 (329, 96, 114, 101)
18015 (328, 96, 114, 100)
19015 (329, 96, 114, 101)
.

In [ ]:
# As an async project to send colours to MQTT

In [158]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [156]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
boardname    esp32colours

Sent 5 lines (115 bytes) to config.txt.


In [ ]:
#%sendtofile main.py

import time, struct
from machine import Pin, I2C
from mqtt_as import config, MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
from machine import Pin, I2C
i2c = I2C(scl=Pin(15), sda=Pin(4), freq=45000)
print(list(map(hex, i2c.scan())))

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
boardname = fconfig["boardname"].encode()    

topiccrgb = boardname+b'/crgb'
client = None

async def TCS34725task():
    i2c.writeto_mem(0x29, 0x80, b'\x03')
    print("chipid", hex(i2c.readfrom_mem(0x29, 0x92, 1)[0]))

    i2c.writeto_mem(0x29, 0x8F, b'\x00')  # gain of 1x
    i2c.writeto_mem(0x29, 0x81, b'\x00')  # integration time 700ms

    i2c.writeto_mem(0x29, 0x8F, b'\x01')  # gain of 2x
    i2c.writeto_mem(0x29, 0x81, b'\xC0')  # integration time 154ms

    i2c.writeto_mem(0x29, 0x8F, b'\x02')  # gain of 4x
    i2c.writeto_mem(0x29, 0x81, b'\xEB')  # integration time 50ms

    i = 0
    while True:
        i += 1
        c,r,g,b = struct.unpack("<HHHH", i2c.readfrom_mem(0x29, 0x94, 8))
        v = "%d %d %d %d"%(c,r,g,b)
        print(v)
        if client and client.isconnected():
            await client.publish(topiccrgb, v)
            pinled.value(i%2)
        else:
            pinled.value((i//20)%2)
        await asyncio.sleep_ms(100)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True)
    
async def mqtttask(bflip=False):
    await asyncio.sleep_ms(5000)
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        print("connecting to:", client._ssid)
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

print("hi there",1)
time.sleep(1)
config['connect_coro'] = onconnecttask
client = MQTTClient(config)
    
print("hi there")
aloop = asyncio.get_event_loop()
aloop.create_task(TCS34725task())
aloop.create_task(mqtttask())
aloop.run_forever()



['0x29']
hi there 1
hi there
chipid 0x44
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
7659 1966 2707 2532
631 162 223 209
630 162 223 208
633 162 224 209
632 162 223 209
630 162 223 208
632 162 224 209
633 162 224 209
630 162 223 208
631 162 223 209
633 162 224 209
631 162 223 208
631 162 223 209
633 162 224 209
632 162 223 209
630 162 223 208
633 162 224 209
632 162 223 209
630 162 223 208
632 162 223 209
633 162 224 209
630 162 223 208
632 162 223 209
633 162 224 209
631 162 223 209
631 162 223 209
631 162 223 209
633 162 224 209
632 162 223 209
630 162 223 208
633 162 224 209
633 162 224 209
630 162 223 208
632 162 223 209
633 162 224 209
630 162 223 208
631 162 223 209
633 162 224 209
631 162 223 208
631 162 223 208
633 162 224 209
connecting to: DoESLiverpool
632 162 223 209
630 162 223 208
632 162 223 209
632 162 223 209
630 162 223 208
632 162 223 209
631 162 223 209
628 161 222 208
629 161 222 208
631 162 223 208
624 161 221 206
605 157 213 199
584 154 205 190
499 133 172 16

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
from machine import Pin, I2C
i2c = I2C(scl=Pin(15), sda=Pin(4))

In [3]:
print(i2c.scan())

[30]


In [4]:
print(hex(30))

0x1e


In [5]:
import math
import machine

from ustruct import pack
from array import array


class HMC5883L:
    __gain__ = {
        '0.88': (0 << 5, 0.73),
        '1.3':  (1 << 5, 0.92),
        '1.9':  (2 << 5, 1.22),
        '2.5':  (3 << 5, 1.52),
        '4.0':  (4 << 5, 2.27),
        '4.7':  (5 << 5, 2.56),
        '5.6':  (6 << 5, 3.03),
        '8.1':  (7 << 5, 4.35)
    }

    def __init__(self, i2c, address=30, gauss='1.3', declination=(0, 0)):
        self.i2c = i2c
        
        # Initialize sensor.
        i2c.start()

        # Configuration register A:
        #   0bx11xxxxx  -> 8 samples averaged per measurement
        #   0bxxx100xx  -> 15 Hz, rate at which data is written to output registers
        #   0bxxxxxx00  -> Normal measurement mode
        i2c.writeto_mem(30, 0x00, pack('B', 0b111000))

        # Configuration register B:
        reg_value, self.gain = self.__gain__[gauss]
        i2c.writeto_mem(30, 0x01, pack('B', reg_value))

        # Set mode register to continuous mode.
        i2c.writeto_mem(30, 0x02, pack('B', 0x00))
        i2c.stop()

        # Convert declination (tuple of degrees and minutes) to radians.
        self.declination = (declination[0] + declination[1] / 60) * math.pi / 180

        # Reserve some memory for the raw xyz measurements.
        self.data = array('B', [0] * 6)

    def read(self):
        data = self.data
        gain = self.gain

        self.i2c.readfrom_mem_into(30, 0x03, data)

        x = (data[0] << 8) | data[1]
        z = (data[2] << 8) | data[3]
        y = (data[4] << 8) | data[5]

        x = x - (1 << 16) if x & (1 << 15) else x
        y = y - (1 << 16) if y & (1 << 15) else y
        z = z - (1 << 16) if z & (1 << 15) else z

        x = round(x * gain, 4)
        y = round(y * gain, 4)
        z = round(z * gain, 4)

        return x, y, z

    def heading(self, x, y):
        heading_rad = math.atan2(y, x)
        heading_rad += self.declination

        # Correct reverse heading.
        if heading_rad < 0:
            heading_rad += 2 * math.pi

        # Compensate for wrapping.
        elif heading_rad > 2 * math.pi:
            heading_rad -= 2 * math.pi

        # Convert from radians to degrees.
        heading = heading_rad * 180 / math.pi
        degrees = math.floor(heading)
        minutes = round((heading - degrees) * 60)
        return degrees, minutes

    def format_result(self, x, y, z):
        degrees, minutes = self.heading(x, y)
        return 'X: {:.4f}, Y: {:.4f}, Z: {:.4f}, Heading: {}° {}′ '.format(x, y, z, degrees, minutes)



In [10]:
import time
for i in range(50):
    h = HMC5883L(i2c)
    print(h.read())
    time.sleep(0.5)

(402.96, 274.16, -574.08)
(611.8, 258.52, -115.0)
(559.36, 320.16, -46.92)
(229.08, 287.96, 186.76)
(5.52, 270.48, 192.28)
(-117.76, 191.36, 160.08)
(-111.32, 185.84, 165.6)
(-110.4, 184.92, 165.6)
(-111.32, 182.16, 165.6)
(-115.0, 180.32, 165.6)
(-230.0, 237.36, 43.24)
(-295.32, 230.92, -11.96)
(-378.12, 449.88, 42.32)
(129.72, 599.84, -88.32)


Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "<stdin>", line 30, in __init__
OSError: [Errno 19] ENODEV


In [13]:
i2c.writeto_mem(0x1E, 0x00, b"\x10")
i2c.writeto_mem(0x1E, 0x01, b"\x20")
i2c.writeto_mem(0x1E, 0x02, b"\x00")


In [43]:
import struct
for i in range(50):
    #i2c.writeto_mem(0x1E, 0x02, b"\x00")
    print(struct.unpack("<hhh", i2c.readfrom_mem(0x1E, 0x03, 6)))
    time.sleep(0.5)


(-20482, 24832, -27136)
(-21250, 24576, -26880)
(-21250, 24576, -27136)
(-22530, 24576, 28416)
(-13314, -29440, 23808)
(-25600, 21505, -3840)
(-26623, -3584, 18689)
(-10239, 23040, -12287)
(-12287, 20992, -7935)
(-15871, 23040, -7679)
(-17919, 24832, -6911)
(-10496, 15873, 13569)
(-26624, 30209, 32512)
(-12288, 30465, 2816)
(20736, 26625, -22273)
(25087, 8961, 4864)
(-6146, -21760, -28160)
(-18690, 24832, -14336)
(-18434, 25600, -12288)
(-18690, 25856, -10752)
(-18690, 26112, -8960)
(-18434, 26368, -9216)
(-18178, 26368, -8192)
(-18946, 25856, -8448)

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)



In [41]:
print(i2c.readfrom_mem(0x1E, 0x02, 1))


b'\x03'
